# Projeto 2 - Ciência dos Dados

• Alunos: Gustavo Guedes, Maia Fleider, Natália Carreras e Tiago Seixas. 

## > Introdução

### ❑ Objetivos e detalhes da base de dados:

▶ O objetivo geral de nosso projeto é realizar um **modelo de predição** de **níveis de radiação solar**. Nesse sentido, analisamos também como outras features (umidade e temperatura, por exemplo) influenciam o comportamento da varíavel. Para a realização do projeto, utlizamos uma base de dados fornecida a participantes da NASA Hackathon (Space Apps Moscow).

▶ A base de dados foi produzida através de dados metereológicos retirados, ao longo de meses, da estação HI-SEAS, localizada no Havaí, Estados Unidos.

<ol>• Para mais detalhes, acesse: 
<li> Kaggle (base de dados): <a href="https://www.kaggle.com/dronio/SolarEnergy" > Solar Radiation </a> </li>
</ol>

<img src="img/painelsolar.jpg" width=550>

## > Minerando Dados e Características do Dataset

In [1]:
#Importando biblotecas úteis:

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import os
import re 

### ❑ Filtros e leitura do Dataset: 

In [2]:
#Iniciando leitura de dados:
raw_data = pd.read_csv('SolarPrediction.csv')
raw_data.head(5)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00


In [3]:
#Filtrando colunas que serão relevantes para futura análise:
dados = raw_data.loc[:,['UNIXTime','Data','Time','Radiation','Temperature','Pressure','Humidity','WindDirection(Degrees)','Speed']]
dados.head(5)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62


In [4]:
#Aplicando funções de filtro e definindo funções auxiliáres:
def cleanup(text):
    punctuation = '[/]' 
    frase = '12:00:00 AM'
    
    pattern = re.compile(punctuation)
    pattern2 = re.compile(frase)
    
    text_subbed = re.sub(pattern, '-', text)
    text_subbed = re.sub(pattern2, '', text)
    
    return text_subbed

def converte_temp(temp):
    
    lista = []
        
    graus = (temp-32)*5/9
        
    lista.append(graus)
        
        
    return lista
    

#Aplicando filtro, somente.
dados.Data = dados.Data.apply(cleanup)

In [5]:
#Trocando '/' por '-' para usar propriedades da biblioteca pandas
dados.head(5)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,1475229326,9/29/2016,23:55:26,1.21,48,30.46,59,177.39,5.62
1,1475229023,9/29/2016,23:50:23,1.21,48,30.46,58,176.78,3.37
2,1475228726,9/29/2016,23:45:26,1.23,48,30.46,57,158.75,3.37
3,1475228421,9/29/2016,23:40:21,1.21,48,30.46,60,137.71,3.37
4,1475228124,9/29/2016,23:35:24,1.17,48,30.46,62,104.95,5.62


In [6]:
#Organizando Data (repare que invertemos a ordem)
dados.Data = pd.to_datetime(dados.Data, dayfirst=True)
dados.head(5)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,1475229326,2016-09-29,23:55:26,1.21,48,30.46,59,177.39,5.62
1,1475229023,2016-09-29,23:50:23,1.21,48,30.46,58,176.78,3.37
2,1475228726,2016-09-29,23:45:26,1.23,48,30.46,57,158.75,3.37
3,1475228421,2016-09-29,23:40:21,1.21,48,30.46,60,137.71,3.37
4,1475228124,2016-09-29,23:35:24,1.17,48,30.46,62,104.95,5.62


In [7]:
#Adicionando colunas de tempo para o dataframe
dados['Year'] = dados['Data'].dt.year
dados['Month'] = dados['Data'].dt.month
dados['Day'] = dados['Data'].dt.day

#Apagando a coluna Data, já que separamos ela
dados_novo = dados.loc[:,['Day','Month','Year','Time','Radiation','Temperature','Pressure',
                          'Humidity','WindDirection(Degrees)','Speed']]

dados_novo.head(5)

,Day,Month,Year,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,29,9,2016,23:55:26,1.21,48,30.46,59,177.39,5.62
1,29,9,2016,23:50:23,1.21,48,30.46,58,176.78,3.37
2,29,9,2016,23:45:26,1.23,48,30.46,57,158.75,3.37
3,29,9,2016,23:40:21,1.21,48,30.46,60,137.71,3.37
4,29,9,2016,23:35:24,1.17,48,30.46,62,104.95,5.62


In [10]:
#Realizados os ajustes, como estamos no Brasil, vamos traduzir e mudar unidades, visando a familiaridade.

dados_novo = dados_novo.rename(columns={"Day": "Dia", "Month": "Mes", "Year":"Ano", "Time":"Hora", 
                                        "Radiation":"Radiacao", "Temperature":"Temperatura", 
                                        "Pressure":"Pressao", "Humidity":"Umidade", "WindDirection(Degrees)":"DirecaoVento",
                                        "Speed":"VelocidadeVento"})

#Aplicando mudanças de unidades:

#Fahrenheit para Graus
lista_graus = dados_novo.Temperatura.apply(converte_temp)

#dados_novos = dados_novo.replace(to_replace = dados_novo.Temperatura, value = lista_graus)

#Milhas por hora para km/hora


#Hg para atm??

dados_novos.head(5)

,Dia,Mes,Ano,Hora,Radiacao,Temperatura,Pressao,Umidade,DirecaoVento,VelocidadeVento
0,29,9,2016,23:55:26,1.21,48,30.46,59,177.39,5.62
1,29,9,2016,23:50:23,1.21,48,30.46,58,176.78,3.37
2,29,9,2016,23:45:26,1.23,48,30.46,57,158.75,3.37
3,29,9,2016,23:40:21,1.21,48,30.46,60,137.71,3.37
4,29,9,2016,23:35:24,1.17,48,30.46,62,104.95,5.62


### ❑ Descrição de variáveis e análise descritiva:

▶ As unidades das colunas de cada dataset são (depois das mudanças, ainda não termindadas):

• Solar radiation: $W/m^2$

• Temperature: °F

• Humidity: %

• Barometric pressure: Hg

• Wind direction: Graus

• Wind speed: Milhas por hora


In [9]:
# Análise exploratória inicial: 
- Análise da Radiação no tempo (Radiação média, por exemplo). 
- Scatter (vulgo gráficos de pontinho de radiação com temperatura, umidade...)

SyntaxError: invalid syntax (<ipython-input-9-76ccb388af4e>, line 2)

### ❑ Storytelling geral dos dados:

## > Modelos de Predição 

### ❑ Descrição e aplicação do Modelo 1 (Treinamento):

### ❑ Descrição e aplicação do Modelo 2 (Treinamento):

## > Processo e Estatísticas de Validação dos modelos (Teste)

### ❑ Validação do Modelo 1:

### ❑ Validação do Modelo 2:

## > Conclusão 

## > Referências Bibliográficas 